In [32]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import json
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [54]:
rental_ratio = pd.read_pickle('housing_rental_ratio.pkl')
rental_ratio = rental_ratio.rename(columns={'RegionName':'zipcode'})
mean_ratio_2015 = np.mean(rental_ratio['ratio_2015'])
mean_pct_delta = np.mean(rental_ratio['ratio_pct_delta'])
rental_ratio['binary_2015'] = rental_ratio['ratio_2015'] > mean_ratio_2015
rental_ratio['zipcode'] = rental_ratio['zipcode'].astype(str)

print 'Mean ratio_2015 is: ' + str(mean_ratio_2015)
rental_ratio.head()

Mean ratio_2015 is: 134.492024044


,zipcode,r_agg_2011,r_agg_2015_adj,r_pct_delta,h_agg_2011,h_agg_2015_adj,h_pct_delta,ratio_2011,ratio_2015,ratio_pct_delta,binary_2015
0,11368,1.868000,2.188278,1.171455,373.833333,431.274105,1.153653,200.124911,197.083754,0.984804,True
1,79936,0.744167,0.694379,0.933096,80.333333,75.060164,0.934359,107.950728,108.096860,1.001354,False
2,60629,1.197000,1.180207,0.985971,107.166667,113.430961,1.058454,89.529379,96.111037,1.073514,False
3,90650,1.524333,1.640026,1.075898,228.750000,292.549418,1.278905,150.065602,178.380911,1.188686,True
4,11226,1.014000,1.277783,1.260141,309.166667,416.876846,1.348389,304.898093,326.250103,1.070030,True


In [53]:
train_set_t = pd.read_pickle('data/train_all_features_norm_and_transform.pkl')
test_set_t = pd.read_pickle('data/test_all_features_norm_and_transform.pkl')
train_set_t['zipcode'] = train_set_t['zipcode'].astype(str)
test_set_t['zipcode'] = test_set_t['zipcode'].astype(str)
print train_set_t.shape, test_set_t.shape

(22122, 64) (5531, 64)


In [75]:
train_merged = pd.merge(train_set_t, rental_ratio, on='zipcode').dropna()
test_merged = test_set_t.merge(rental_ratio, on='zipcode').dropna()
print train_merged.shape, test_merged.shape

(7761, 74) (1895, 74)


In [76]:
'''lcols = ['A002',
         'A054',
 'A003',
 'median_income',
 'A050',
 'A052',
 'A024',
 'A051',
 'A038',
 'A044',
 'A035',
 'A026',
 'A036',
 'A055',
 'A040',
 'A029',
 'A048',
 'A017',
 'A037',
 'A042',
 'A020',
 'A047',
 'A046',
 'A056',
 'A033',
 'A025',
 'n_establishments',
 'paid_employees',
 'A034',
 'A045',
 'A028',
 'prison',
 'first_quarter_payroll_1000',
 'A009',
 'A019',
 'A032',
 'A031',
 'A041',
 'A039',
 'annual_payroll_1000',
 'A030',
 'A043',
 'A049',
 'zipcode',
 'ratio_2011']'''

In [88]:
#lcols = [feature if not in ['Intercept', 'response'] for feature in best_features] + ['ratio_2011']

Xmatrix_train = train_merged[lcols].values
Yresp_train = train_merged['binary_2015'].values.astype(str)
Xmatrix_test = test_merged[lcols].values
Yresp_test = test_merged['binary_2015'].values.astype(str)

In [89]:
from sklearn.grid_search import GridSearchCV
from sklearn.svm import LinearSVC

clfsvm=LinearSVC(loss="hinge")
Cs=[0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

gs = GridSearchCV(clfsvm, param_grid=dict(C=Cs), cv=5, scoring="accuracy")
fit = gs.fit(Xmatrix_train,Yresp_train)
best = fit.best_params_
print "BEST", best

BEST {'C': 0.01}


In [90]:
#calculate the accuracy here
#your code here
gsBetter = GridSearchCV(clfsvm,param_grid=dict(C=best.values()), cv=5, scoring="accuracy")
fitBetter = gs.fit(Xmatrix_train,Yresp_train)
trainingAccuracy = fitBetter.score(Xmatrix_train,Yresp_train)
testAccuracy = fitBetter.score(Xmatrix_test,Yresp_test)
print "BEST", gs.best_params_
print "############# based on standard predict ################"
print "Accuracy on training data: %0.5f" % (trainingAccuracy)
print "Accuracy on test data:     %0.5f" % (testAccuracy)

BEST {'C': 0.1}
############# based on standard predict ################
Accuracy on training data: 0.62724
Accuracy on test data:     0.62850


//anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:370: ChangedBehaviorWarning: The long-standing behavior to use the estimator's score function in GridSearchCV.score has changed. The scoring parameter is now used.
  ChangedBehaviorWarning)
